# Visualise Trajectory Profiles

In [ ]:
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
%matplotlib inline

from pru.EcefPoint import rad2nm
from pru.EcefPath import EcefPath
from pru.ecef_functions import calculate_EcefPoints
from pru.trajectory_functions import calculate_elapsed_times, calculate_speed
from pru.horizontal_path_functions import derive_horizontal_path
from pru.trajectory_analysis import DEFAULT_ACROSS_TRACK_TOLERANCE, \
    calculate_ground_speeds, smooth_times

## Get positions file
Examples:   
286375_cpr_positions_2017-07-01.csv  
286757_cpr_positions_2017-07-01.csv  
287041_cpr_positions_2017-07-01.csv  

In [ ]:
filename = input('filename: ')
points_df = pd.read_csv(filename, parse_dates=['TIME'])
len(points_df)

## Plot the positions

In [ ]:
# Get the point coordinates as a nested list from the points_df dataframe
point_locations = points_df[['LAT','LON']].values.tolist()

In [ ]:
# Create the map
map_mb = folium.Map(location=[50.0, 10.0], zoom_start=5, tiles='Cartodb Positron')

In [ ]:
BLACK = '#000000'
BLUE = '#0000FF'
RED = '#FF0000'

# Add a PolyLine to show point_locations
folium.PolyLine(
    locations=point_locations,
    color=BLACK,
    weight=4,
    tooltip=filename,
).add_to(map_mb)

In [ ]:
# Display the map
map_mb

## Calculate the Horizontal Path

In [ ]:
ecef_points = calculate_EcefPoints(points_df['LAT'].values,
                                   points_df['LON'].values)

In [ ]:
threshold = 2 * DEFAULT_ACROSS_TRACK_TOLERANCE

In [ ]:
threshold=np.deg2rad(threshold / 60.0)
ecef_path = derive_horizontal_path(ecef_points,
                                   threshold=threshold)
len(ecef_path)

In [ ]:
ecef_path_distances = rad2nm(ecef_path.path_distances())
ecef_path_length = ecef_path_distances[-1]
ecef_path_length

## Plot Horizontal Path

In [ ]:
waypoint_locations = []
for point in ecef_path.points:
    waypoint_locations.append(point.to_lat_long())
#waypoint_locations

In [ ]:
# Add a PolyLine to show point_locations
folium.PolyLine(
    locations=point_locations,
    color=RED,
    weight=2,
    tooltip='raw points',
).add_to(map_mb)

# Add a PolyLine to show waypoint_locations
folium.PolyLine(
    locations=waypoint_locations,
    color=BLUE,
    weight=2,
    tooltip='extreme points',
).add_to(map_mb)

In [ ]:
# Display the map
map_mb

## Sort Points

Calculate distances along the horizontal path. 

Create a new "sorted" Data frame, sorted by this distance and time.  
Determine whether the sort order has changed.

In [ ]:
path_distances = rad2nm(ecef_path.calculate_path_distances(ecef_points, threshold))
len(path_distances)

In [ ]:
path_length = path_distances[-1]
path_length

In [ ]:
# Sort positions by path distance then time
sorted_df = pd.DataFrame({'distance': path_distances,
                          'time': points_df['TIME'].values,
                          'altitude': points_df['ALT'].values,
                          'points': ecef_points})
sorted_df.sort_values(by=['distance', 'time'], inplace=True)

In [ ]:
# determine whether the position order has changed
sorted_path_distances = sorted_df['distance'].values
unordered = (path_distances != sorted_path_distances)
unordered.any()

## Display Vertical Profile

Plot the vertical profile as altiude vs path distance.

In [ ]:
alts = sorted_df['altitude'].values
len(alts)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.set(title='Vertical Profile',
       xlabel='Path Distance (NM)',
       ylabel='Altitude (feet)')
plt.plot(sorted_path_distances, alts, 'o-')
for i in range(len(sorted_path_distances)):
    ax.annotate(str(i), (sorted_path_distances[i], alts[i] +500))
plt.show

## Calculate Ground Speeds

Calculate ground speeds between positions from the 
path distances and elapsed times from the first position.

In [ ]:
times = sorted_df['time'].values
elapsed_times = calculate_elapsed_times(times, times[0])
len(elapsed_times)

In [ ]:
speeds = calculate_ground_speeds(sorted_path_distances, elapsed_times, 120)  
len (speeds)

## Plot Ground Speeds

Plot ground speeds vs path distance.

Note: the first speed is always zero.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.set(title='Speed Profile',
       xlabel='Path Distance (NM)',
       ylabel='Ground Speed (Knots)')
plt.plot(sorted_path_distances, speeds, 'o-')
for i in range(len(sorted_path_distances)):
   ax.annotate(str(i), (sorted_path_distances[i], speeds[i] +10))
plt.show

## Smooth Times and Ground Speeds

Use the smooth_times to smooth the elapsed_times.

In [ ]:
smoothed_times = smooth_times(sorted_path_distances, elapsed_times, 5, 3, 120.0)
len(smoothed_times)

In [ ]:
# Calculate differences from elapsed_times
delta_times = smoothed_times - elapsed_times
# delta_times

### Calculate the mean time difference

In [ ]:
mean_delta = np.sum(delta_times) / len(delta_times)
mean_delta

### Calculate the time Standard Deviation

In [ ]:
# Offset the smoothed_times by the mean time difference
smoothed_times = smoothed_times - mean_delta

sq_delta_times = (smoothed_times - elapsed_times) ** 2
time_sd = np.sqrt(np.sum(sq_delta_times) / (len(sq_delta_times) - 1))
time_sd

In [ ]:
leg_lengths = np.ediff1d(sorted_path_distances, to_begin=[0])
smoothed_durations = np.ediff1d(smoothed_times, to_begin=[0])
smoothed_speeds = calculate_speed(leg_lengths, smoothed_durations)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.set(title='Smoothed Speed Profile',
       xlabel='Path Distance (NM)',
       ylabel='Ground Speed (Knots)')
plt.plot(sorted_path_distances, smoothed_speeds, 'o-')
for i in range(len(sorted_path_distances)):
    ax.annotate(str(i), (sorted_path_distances[i], smoothed_speeds[i] +10))
plt.show